In [70]:
import pandas as pd
from gensim.models import FastText
import re
import numpy as np
from libsvm import svmutil
import libsvm

In [71]:
data = pd.read_excel("NewsHeadlines_01012019_30092019.xlsx")
# ls = [1 if i < 4761 else 0 for i in range(47611)]
# index = data["Like Count"].tolist()
data["Category"].replace({"Tech": "Technology","-":"Uncategorized", "Sports":"Sport"}, inplace=True)
data.head()

,id,Page,Date,Time,Like Count,Comment Count,Share Count,Category,Crime_flag,Feed_Img,Headline,Summary,Publish Timestamp,Post Message,Post Description,Post Link,Post Type,Post Img,Status Type
0,1,The Straits Times,01-01-2018 00:28:04,2,770,46,153,Main,0,0,Six-minute display of fireworks,What better way to usher in 2018 than with a f...,01-01-2018 00:28:04,What better way to usher in 2018 than with a f...,-,https://www.facebook.com/TheStraitsTimes/video...,video,0,added_video
1,2,The Straits Times,01-01-2018 02:05:27,5,294,30,218,Main,1,1,Singaporean man killed in motorcycle accident ...,SINGAPORE - A pair of Singaporeans believed to...,01-01-2018 01:33:52,The accident occurred near Thailand's southern...,SINGAPORE - A pair of Singaporeans believed to...,http://www.straitstimes.com/singapore/singapor...,link,0,shared_story
2,3,The Straits Times,01-01-2018 04:04:20,9,23,2,6,Asia,0,1,Hundreds of Indonesian couples ring in the new...,JAKARTA (REUTERS) - Hundreds of Indonesian cou...,01-01-2018 00:02:23,Nearly 450 couples gathered in a large tent in...,JAKARTA (REUTERS) - Hundreds of Indonesian cou...,http://www.straitstimes.com/asia/se-asia/hundr...,link,0,shared_story
3,4,The Straits Times,01-01-2018 04:05:45,9,27,13,1,Sport,0,1,Football: Pogba urges misfiring Man United to ...,LONDON (AFP) - Paul Pogba has called on Manche...,31-12-2017 23:03:53,"'We drew again. Three draws, we have to win, s...",LONDON (AFP) - Paul Pogba has called on Manche...,http://www.straitstimes.com/sport/football/foo...,link,0,shared_story
4,5,The Straits Times,01-01-2018 04:07:02,9,26,5,4,Sport,0,1,Football: Nike advertises Coutinho Barcelona s...,PARIS (AFP) - An apparent gaffe by Nike has fi...,31-12-2017 23:48:51,"Headlined 'Where the magic happens', the page,...",PARIS (AFP) - An apparent gaffe by Nike has fi...,http://www.straitstimes.com/sport/football/foo...,link,0,shared_story


In [4]:
train_df = data["Summary"]

In [7]:
train_df = train_df.values.tolist()

In [14]:
summary = []
for sent_str in train_df:
#     sent_str = re.sub(r"'", "", str(sent_str).lower())
    tokens = re.sub(r"[^a-z0-9]+", " ", str(sent_str).lower()).split()
    summary.append(tokens)
summary

In [3]:
model_summary = FastText(summary , size=100, window=5, min_count=5, workers=4,sg=1)

NameError: name 'summary' is not defined

In [20]:
model_summary.wv.most_similar("indian")

In [4]:
# model_summary.save("model_summary.bin")
model_summary = FastText.load("model_summary.bin")

In [72]:
category = set(data["Category"].values.tolist())

In [73]:
category

{'1819-C',
 'AHTC',
 'Abroad',
 'Abroad-C',
 'Almost Famous',
 'Anyhowly Driving',
 'Art',
 'Asia',
 'Asia Pacific',
 'Business',
 'CNA Insider',
 'Celebrity',
 'Commentary',
 'Community',
 'Culture',
 'Drama',
 'Entertainment',
 'Events',
 'Fake News',
 'Health',
 'Heartwarming',
 'History',
 'Interviews',
 'LifeStyle',
 'Lifestyle',
 'Main',
 'News',
 'Offbeat',
 'Parliament',
 'Personal Finance',
 'Perspectives',
 'Singapore',
 'Singapore Angst',
 'Singapore Budget 2018',
 'Singapore Parliament',
 'Social Media Fail',
 'Sponsored Content',
 'Sport',
 'Technology',
 'Things To Do',
 'Trending',
 'Uncategorized',
 'Weird AF',
 'World'}

In [74]:
headlines= []
for sent_str in data["Headline"]:
#     sent_str = re.sub(r"'", "", str(sent_str).lower())
    tokens = re.sub(r"[^a-z0-9]+", " ", str(sent_str).lower()).split()
    if(tokens == []):
        tokens = "nan"
    headlines.append(tokens)

In [75]:
cat_vec = {}
for c in category:
    cat_vec[c] = model_summary[c]

/home/prachi/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [76]:
head_vec = []
n = len(headlines)
for h in range(n):
    sum1 = np.array([0.0 for i in range(100)])
    for tok in headlines[h]:
        sum1+=model_summary[tok]
#     print(sum1)
    sum1 = sum1/len(headlines[h])
#     print(sum1)
    sum1 = np.concatenate((sum1,cat_vec[data["Category"][h]]))
    head_vec.append(sum1)


/home/prachi/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [10]:
# head_vec[0]

In [11]:
#  head_vec[:4261]

In [36]:
help(svmutil)

Help on module libsvm.svmutil in libsvm:

NAME
    libsvm.svmutil

CLASSES
    _ctypes.PyCFuncPtr(_ctypes._CData)
        ctypes.CFunctionType
    _ctypes.Structure(_ctypes._CData)
        svm.svm_model
        svm.svm_node
        svm.svm_parameter
        svm.svm_problem
    _ctypes._SimpleCData(_ctypes._CData)
        ctypes.c_double
    
    PRINT_STRING_FUN = class CFunctionType(_ctypes.PyCFuncPtr)
     |  Function Pointer
     |  
     |  Method resolution order:
     |      CFunctionType
     |      _ctypes.PyCFuncPtr
     |      _ctypes._CData
     |      builtins.object
     |  
     |  Data descriptors defined here:
     |  
     |  __dict__
     |      dictionary for instance variables (if defined)
     |  
     |  __weakref__
     |      list of weak references to the object (if defined)
     |  
     |  ----------------------------------------------------------------------
     |  Methods inherited from _ctypes.PyCFuncPtr:
     |  
     |  __bool__(self, /)
     |      sel

In [126]:
train = head_vec[:int(0.8*(47611))]
res_train = list(np.log2(np.array(data["Like Count"][:int(0.8*47611)].tolist())+1))
test = head_vec[int(0.8*47611):]
res_test = list(np.log2(np.array(data["Like Count"][int(0.8*47611):].tolist())+1))
print(len(train), len(res_train))

38088 38088


In [149]:
model_svm_reg = svmutil.svm_train(res_train, train,'-s 4 -v 10 -t 2 -c 2')

Cross Validation Mean squared error = 2.90463
Cross Validation Squared correlation coefficient = 0.168539


In [78]:
# res_train[31000:31100]

In [127]:
model_svm = svmutil.svm_train(res_train, train,'-s 4 -t 2 -c 2')

In [128]:
y_reg = svmutil.svm_predict(res_test+res_train, test+train, model_svm)

Mean squared error = 3.41269 (regression)
Squared correlation coefficient = 0.127473 (regression)


In [148]:
len(y_reg[0])

47611

In [139]:
svmutil.svm_save_model("model_svm_reg.bin",model_svm)

In [102]:
from sklearn.metrics import r2_score, mean_absolute_error

In [143]:
r2 = r2_score(res_test+res_train,y_reg[0])
r2

0.11785056365054625

In [103]:
# res_test

In [132]:
mae = mean_absolute_error(res_test+res_train, y_reg[0])
mae

1.4486094132772414

In [133]:
from scipy.stats import kendalltau
kt = kendalltau(res_test+res_train, y_reg[0])
kt

KendalltauResult(correlation=0.2449858856993867, pvalue=0.0)

In [120]:
y_reg2 = svmutil.svm_predict(res_train, train, model_svm)

Mean squared error = 8.37172 (regression)
Squared correlation coefficient = 0.178916 (regression)


In [134]:
r2 = r2_score(res_train,y_reg2[0])
r2

-0.7402639696723965

In [135]:
mae = mean_absolute_error(res_train, y_reg2[0])
mae

1.9817713724319508

In [136]:
kt = kendalltau(res_train, y_reg2[0])
kt

KendalltauResult(correlation=0.2806766116648065, pvalue=0.0)

In [137]:
y_reg3 = svmutil.svm_predict(res_test, test, model_svm)

Mean squared error = 5.5019 (regression)
Squared correlation coefficient = 0.106138 (regression)


In [138]:
r2 = r2_score(res_test,y_reg3[0])
r2

-0.21837228661490915

In [140]:
mae = mean_absolute_error(res_test, y_reg3[0])
mae

1.8209704151582984

In [142]:
from scipy.stats import kendalltau
kt = kendalltau(res_test, y_reg3[0])
kt

KendalltauResult(correlation=0.20718250640494856, pvalue=1.63502469638562e-200)